In [229]:
import pandas as pd
import os
import numpy as np


## WIDER face

In [230]:
mat_file_path = '/Users/Study/Downloads/WIDER/wider_face_split/wider_face_train.mat'
mat_train = scipy.io.loadmat(mat_file_path)

print(mat_train.keys())

dict_keys(['__header__', '__version__', '__globals__', 'blur_label_list', 'event_list', 'expression_label_list', 'face_bbx_list', 'file_list', 'illumination_label_list', 'invalid_label_list', 'occlusion_label_list', 'pose_label_list'])


In [231]:
file_list = mat_train['file_list']
face_list = mat_train['face_bbx_list']

In [232]:
names_bboxes = {}

for index in range(len(file_list)):
  counter = -1
  for img_obj in file_list[index][0]:
    counter += 1
    params_list = []
    for params in face_list[index][0][counter][0]:
      params[0] += params[2]/2
      params[1] += params[3]/2
      params_list.append(np.array(params))
    names_bboxes[img_obj[0][0]] = (params_list)

In [233]:
names_images_dims = [[], [], []]

base_path = '/Users/Study/Downloads/WIDER/WIDER_train_full'

for img_name in os.listdir(base_path):
  img_path = os.path.join(base_path, img_name)
  img = cv2.imread(img_path)
  if img is not None:
    original_height, original_width = img.shape[:2]
    coeff = original_height/original_width
    base = 240
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (int(base), int(base*coeff)))
    names_images_dims[0].append(img_name[:-4])
    names_images_dims[1].append(img/255.0)
    names_images_dims[2].append([float(original_width), float(original_height)])

In [234]:
images_bboxes_train = {}
for name in names_images_dims[0]:
  images_bboxes_train[name] = names_images_dims[1][names_images_dims[0].index(name)], names_bboxes[name], names_images_dims[2][names_images_dims[0].index(name)]

In [235]:
for key in images_bboxes_train.keys():
  counter = -1
  for faces in images_bboxes_train[key][1]:
    counter+=1
    images_bboxes_train[key][1][counter] = images_bboxes_train[key][1][counter].astype(np.float32)
    images_bboxes_train[key][1][counter][0] = faces[0]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][1] = faces[1]/images_bboxes_train[key][2][1]
    images_bboxes_train[key][1][counter][2] = faces[2]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][3] = faces[3]/images_bboxes_train[key][2][1]

## Write formatted

In [236]:
keys = list(images_bboxes_train.keys())

In [237]:
val_dict = {}
for key in range(150):
  val_dict[keys[key]] = images_bboxes_train[keys[key]]
  images_bboxes_train.pop(keys[key])

In [238]:
keys = list(images_bboxes_train.keys())

In [240]:
for key in range(8000):
  images_bboxes_train.pop(keys[key])

In [241]:
imroot = '/Users/Study/Downloads/WIDER/root/images/train/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/train/'

for key in images_bboxes_train.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(images_bboxes_train[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in images_bboxes_train.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "0 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n")  

In [242]:
imroot = '/Users/Study/Downloads/WIDER/root/images/val/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/val/'

for key in val_dict.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(val_dict[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in val_dict.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "0 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n") 

## MAFA dataset

In [243]:
mat_file_path_mafa = '/Users/Study/Downloads/MAFA/MAFA-Label-Train/LabelTrainAll.mat'
mafa_mat = scipy.io.loadmat(mat_file_path_mafa)


print(mafa_mat.keys())

dict_keys(['__header__', '__version__', '__globals__', 'label_train'])


In [244]:
file_list = []

for count in range(len(mafa_mat['label_train'][0])):
  file_list.append(mafa_mat['label_train'][0][count][1][0][:-4])

In [245]:
face_list = []

for count in range(len(mafa_mat['label_train'][0])):
  picture = []
  for faces in mafa_mat['label_train'][0][count][2]:
    face = []
    for coords in range(4):
      face.append(faces[coords])
    picture.append(face)
  face_list.append(picture)

In [246]:
names_bboxes = {}

counter = -1
for img_obj in file_list:
  counter += 1
  params_list = []
  for params in face_list[counter]:
    params_list.append(np.array(params))
  names_bboxes[img_obj] = (params_list)

In [247]:
names_images_dims = [[], [], []]

base_path = '/Users/Study/Downloads/MAFA/train-images'

for img_name in os.listdir(base_path):
  img_path = os.path.join(base_path, img_name)
  img = cv2.imread(img_path)
  if img is not None:
    original_height, original_width = img.shape[:2]
    coeff = original_height/original_width
    base = 240
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (int(base), int(base*coeff)))
    names_images_dims[0].append(img_name[:-4])
    names_images_dims[1].append(img/255.0)
    names_images_dims[2].append([float(original_width), float(original_height)])

libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: cHRM chunk does not match sRGB
libpng warning: iCCP: known incorrect sRGB profile


In [248]:
images_bboxes_train = {}
for name in names_images_dims[0]:
  images_bboxes_train[name] = names_images_dims[1][names_images_dims[0].index(name)], names_bboxes[name], names_images_dims[2][names_images_dims[0].index(name)]

In [249]:
for key in images_bboxes_train.keys():
  counter = -1
  for faces in images_bboxes_train[key][1]:
    counter+=1
    images_bboxes_train[key][1][counter] = images_bboxes_train[key][1][counter].astype(np.float32)
    images_bboxes_train[key][1][counter][0] = faces[0]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][1] = faces[1]/images_bboxes_train[key][2][1]
    images_bboxes_train[key][1][counter][2] = faces[2]/images_bboxes_train[key][2][0]
    images_bboxes_train[key][1][counter][3] = faces[3]/images_bboxes_train[key][2][1]

In [250]:
keys = list(images_bboxes_train.keys())

In [251]:
val_dict = {}
for key in range(150):
  val_dict[keys[key]] = images_bboxes_train[keys[key]]
  images_bboxes_train.pop(keys[key])

In [252]:
keys = list(images_bboxes_train.keys())

In [254]:
for key in range(21000):
  images_bboxes_train.pop(keys[key])

In [255]:
imroot = '/Users/Study/Downloads/WIDER/root/images/train/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/train/'

for key in images_bboxes_train.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(images_bboxes_train[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in images_bboxes_train.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "1 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n")  

In [256]:
imroot = '/Users/Study/Downloads/WIDER/root/images/val/'
lbroot = '/Users/Study/Downloads/WIDER/root/labels/val/'

for key in val_dict.keys():
  file_path = os.path.join(imroot, f"{key}.jpg")
  image = np.clip(val_dict[key][0] * 255.0, 0, 255).astype(np.uint8)
  cv2.imwrite(file_path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

for key, values in val_dict.items():
  file_path = os.path.join(lbroot, f"{key}.txt")
  with open(file_path, 'w') as file:
    for bbox in values[1]:
      bbox_str = "1 " + " ".join(map(str, bbox))
      file.write(bbox_str + "\n") 

In [ ]:
model = YOLO("yolov8n.yaml")
model.to('cuda')

In [ ]:
res = model.train(data='faces.yaml', verbose=True, epochs=10, amp=False, batch=12)